Criar um conjunto de filmes que possa ser oferecido a determinado usuário

# 0. Importar as bibliotecas

In [22]:
import pandas as pd
import matplotlib.pyplot as plt #plotar gráficos no estilo matlab
import seaborn as sns #plotar gráficos com mais detalhes

# 1. Importar os arquivos

In [10]:
df_filmes = pd.read_csv ('data/movies.csv')
df_notas = pd.read_csv ('data/ratings.csv')

# 2. Analisar os datasets

## 2.1 Visualizar parte dos datasets

In [ ]:
#Dataset sobre os filmes
df_filmes.head()

In [ ]:
#Dataset sobre nota dos filmes
df_notas.head()

## 2.2 Combinando os datasets

In [ ]:
#Merge entre os datasets para visualizar os filmes vistos por cada usuário
filmes_notas = pd.merge(df_filmes, df_notas, on = 'movieId')
filmes_notas.head()


## 2.3 Média de avaliação dos filmes

In [ ]:
filmes_notas.groupby('title')['rating'].mean().sort_values( ascending = False ).head()

## 2.4 Total de notas que cada filme recebeu
#### A nota média não é o ideal para cada filme, porque há filmes que há apenas uma nota atribuída para cada filme. portanto é necessário avaliar a quantidade de avaliações de cada filme

In [2]:
#Ver alguns filmes com nota 1
filmes_notas.groupby('title')['rating'].count().sort_values( by = ['hour'], ascending = True ).head()

NameError: name 'filmes_notas' is not defined

In [ ]:
#Ver os filmes com maior quantidade de avaliações 
#Teste
filmes_notas.groupby('title')['rating'].count().sort_values( ascending = False ).head()

## 2.5 Criar um dataframe com a média de avaliação de cada filme

In [ ]:
med_aval_filmes = pd.DataFrame( filmes_notas.groupby('title')['rating'].mean() ) 
med_aval_filmes.sort_values( by = 'rating', ascending = False ).head()

## 2.6 Adicionar uma coluna no novo dataframe com a quantidade de avaliações recebidas 

In [ ]:
med_aval_filmes['n_notas'] = pd.DataFrame( filmes_notas.groupby('title')['rating'].count() ) 
med_aval_filmes.sort_values( by = 'n_notas', ascending = False ).head()

## 2.7 Boxplot da média das avaliações recebidas pelos filmes

In [ ]:
med_aval_filmes.boxplot( column='rating' )

## 2.8 Histograma da média das avaliações recebidas pelos filmes

In [ ]:
plt.figure( figsize=(10,4) )
med_aval_filmes['rating'].hist( bins=70 )

# 3. Construção do modelo de recomendação

## 3.1 Criar uma matriz pivô das notas que cada usuário atribuiu a cada filme

In [ ]:
filmes_notas_matriz = filmes_notas.pivot_table( index='userId', columns='title', values='rating' )
filmes_notas_matriz.head()

## 3.2 Criar uma matriz de correlação entre os filmes

In [ ]:
filmes_corr_matriz = filmes_notas_matriz.corr()
filmes_corr_matriz.head()
#Alguns filmes realmente vão ficar com valor NaN porque não tiveram avaliação

## 3.3 Refinar a matriz de correlação entre os filmes

In [ ]:
filmes_corr_matriz = filmes_notas_matriz.corr( method='pearson', min_periods=50 ) #Apenas filmes que tiveram mais de 50 avaliações
filmes_corr_matriz.head()


## 3.4 Primeiro teste com usuário

### 3.4.1 Teste com apenas 1 filme de 1 usuário

In [ ]:
# cria uma variável com as notas que o usuário 600 atribuiu aos filmes
test_usuario = filmes_notas_matriz.iloc[600].dropna()
test_usuario.head(10).sort_values( ascending=False )

In [87]:
# identifica o filme de índice 2 
test_usuario.index[2]

'Armageddon (1998)'

In [ ]:
# Verificar as correlações não nulas do filmes de índice 2 (Armagedon)
# Aqui pode verificar o quanto o filme Armagedom se assemelha (de alguma forma) com os demais, dessa forma podendo recomendar os de maior correlação positiva

filmes_corr_matriz[test_usuario.index[2]].dropna().sort_values( ascending=False )

### 3.4.2 Teste com todos os filmes assistidos pelo usuário 600

In [ ]:
#Adicionar filmes similares
filmes_similares_candidatos = pd.Series()
for i in range ( 0, len(test_usuario.index) ):
    print( "Adicionando filme similiar a " +test_usuario.index[i]+ "..." )
    #recupera o filme que apresenta a correlação
    similar = filmes_corr_matriz[test_usuario.index[i]].dropna() #Pega a lista de filmes que o usuário já viu e retira os valores nulos
    similar = similar.map( lambda x: x * test_usuario[i] ) #Multiplica o valor da correlação pela nota que o usuário deu para o filme para deixar os filmes na mesma escala
    filmes_similares_candidatos  =filmes_similares_candidatos.append(similar)


In [106]:
#Ordena os filmes similares

filmes_similares_candidatos.sort_values(inplace=True,  ascending=False )

In [ ]:
#Como o mesmo filme pode ter similaridade com mais de um, soma-se a similaridade deles (as notas)
"""filmes_similares_candidatos = filmes_similares_candidatos.groupby( filmes_similares_candidatos.index ).sum()
filmes_similares_candidatos.sort_values( inplace=True, ascending=False )
filmes_similares_candidatos.head()"""



filmes_similares_candidatos=filmes_similares_candidatos.groupby(filmes_similares_candidatos.index).sum()
filmes_similares_candidatos.sort_values(inplace=True,ascending=False)
filmes_similares_candidatos.head(10)

In [ ]:
#Dentro dos filmes candidatos, exclui os filmes que já foram assistidos pelo usuário
filtra_filme = filmes_similares_candidatos[~filmes_similares_candidatos.isin(test_usuario.index)]
filtra_filme.head(10)

In [ ]:
filtra_movies=filmes_similares_candidatos[~filmes_similares_candidatos.isin(test_usuario.index)]
filtra_movies.head()

In [ ]:
#Filmes que seriam recomendados
filtra_movies_recomendados=filtra_movies.sort_values(ascending=False)
filtra_movies_recomendados.head()